# Support Vector Machines

Here we explore the usage of SVM

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

In [3]:
from sklearn.model_selection import train_test_split
raw_all_features_df = pd.read_csv("training_set_features.csv", index_col="respondent_id")
all_labels_df = pd.read_csv("training_set_labels.csv", index_col="respondent_id")

test_set_features_df = pd.read_csv("test_set_features.csv", index_col="respondent_id")

submission_df = pd.read_csv("submission_format.csv", index_col="respondent_id")

raw_train_features_df, raw_test_features_df, train_labels_df, test_labels_df = train_test_split(
    raw_all_features_df,
    all_labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=all_labels_df,
    random_state=25519
)

raw_train_features_df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1737,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,1.0,2.0,5.0,2.0,2.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,dqpwygqj,"MSA, Not Principle City",2.0,0.0,fcxhlnwr,mxkfnird
24400,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,2.0,4.0,4.0,4.0,4.0,35 - 44 Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,atmpeygn,"MSA, Principle City",0.0,0.0,pxcmvdjn,mxkfnird
21693,2.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,5.0,4.0,2.0,4.0,5.0,2.0,35 - 44 Years,College Graduate,Hispanic,Female,"> $75,000",Married,Own,Employed,mlyzmhmf,"MSA, Principle City",1.0,3.0,wxleyezf,emcorrxb
20247,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,2.0,2.0,5.0,2.0,2.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,oxchjgsf,Non-MSA,0.0,0.0,cfqqtusy,emcorrxb
3172,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,3.0,2.0,5.0,4.0,2.0,65+ Years,NaN,Other or Multiple,Male,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,mlyzmhmf,"MSA, Principle City",0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14128,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,5.0,2.0,2.0,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,kbazzjca,"MSA, Not Principle City",1.0,0.0,NaN,NaN
14882,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,2.0,5.0,4.0,1.0,55 - 64 Years,College Graduate,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",0.0,0.0,wxleyezf,emcorrxb
16187,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,55 - 64 Years,NaN,Other or Multiple,Male,NaN,NaN,NaN,NaN,oxchjgsf,"MSA, Principle City",0.0,0.0,NaN,NaN


In [7]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
def ordinalize(features: pd.DataFrame, raw: pd.DataFrame, name: str) -> pd.DataFrame:
    ordinal_arr = OrdinalEncoder().fit_transform(raw[[name]])
    
    features.drop(name, axis=1)
    features[name] = ordinal_arr
    return features

    
def onehotize(features: pd.DataFrame, raw: pd.DataFrame, name: str) -> pd.DataFrame:
    enc = OneHotEncoder(sparse=False)
    onehot_df = pd.DataFrame(
        enc.fit_transform(raw[name]),
        columns=enc.get_feature_names_out(),
    )
    onehot_df.index = raw.index
    return pd.concat(
        [features.drop(name, axis=1), onehot_df],
        axis=1,
    )


def normalize(in_df: pd.DataFrame) -> pd.DataFrame:
    out_df = in_df
    out_df = onehotize(out_df, in_df, ["employment_industry", "employment_occupation", "race", "employment_status", "hhs_geo_region", "census_msa"])
    out_df = ordinalize(out_df, in_df, "age_group")
    out_df = ordinalize(out_df, in_df, "education")
    out_df = ordinalize(out_df, in_df, "sex")
    out_df = ordinalize(out_df, in_df, "income_poverty")
    out_df = ordinalize(out_df, in_df, "marital_status")
    out_df = ordinalize(out_df, in_df, "rent_or_own")
    # out_df = StandardScaler().fit_transform(out_df.fillna(out_df.mean()))
    # return DBSCAN(eps=10).fit(out_df).labels_
    return StandardScaler().fit_transform(out_df.fillna(out_df.mean()))

train_features_df = normalize(raw_train_features_df)

test_features_df = normalize(raw_test_features_df)

train_features_df

array([[-0.67232049,  1.19848465, -0.22654295, ...,  1.141027  ,
        -0.64943972, -0.6065161 ],
       [-0.67232049, -2.04585573, -0.22654295, ..., -0.87640345,
         1.53978879, -0.6065161 ],
       [ 0.42839595,  1.19848465,  4.42579937, ..., -0.87640345,
         1.53978879, -0.6065161 ],
       ...,
       [-0.67232049,  1.19848465, -0.22654295, ..., -0.87640345,
         1.53978879, -0.6065161 ],
       [-0.67232049,  1.19848465, -0.22654295, ...,  1.141027  ,
        -0.64943972, -0.6065161 ],
       [ 1.52911239,  1.19848465, -0.22654295, ...,  1.141027  ,
        -0.64943972, -0.6065161 ]])

In [34]:
%%time
from sklearn.svm import SVC
from sklearn.cluster import DBSCAN
from sklearn.multioutput import MultiOutputClassifier

svclassifier = MultiOutputClassifier( SVC(random_state=31415, kernel='linear', probability=True))
svclassifier.fit(train_features_df, train_labels_df)

In [ ]:
preds = svclassifier.predict(test_features_df)

preds_df = pd.DataFrame({"h1h1_vaccine": preds[:, 0], "seasonal_vaccine": preds[:, 1]}, index=test_labels_df.index)
preds_df

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_labels_df, preds_df)